# NLPHW1 - Named Entity Recognition
Francesco Starna
1613660

## Setup
Imports.

In [ ]:
import torch
from torch.utils.data import IterableDataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torch.autograd as autograd
import torch.optim as optim
from torch.nn.utils import clip_grad_value_

import nltk
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')

from collections import Counter
import csv, pickle

torch.manual_seed(1)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Data Preparation

### Vocabulary
The class vocabulary it has been thought to create a general vocabulary where each word is lowered and lemmatize. nltk WordNet Lemmatizer is used to get lemma from plurals and verbs.

In [ ]:
class Vocab():

  def __init__(self, vocab_size, unk_token='<unk>', 
               path=None, tag_to_index=None, words=None, characters=None):

    self.unk_token = unk_token
    self.word_vocab_size = vocab_size[0]
    self.char_vocab_size = vocab_size[1]

    if path is None:
      self.__word_vocabulary, self.__char_vocabulary = words, characters
      self.tag_to_index = tag_to_index
    
    else:
      self.tag_to_index = {'O':0, 'PER':1, 'ORG':2, 'LOC':3, '<pad>':9, '<test>': 8}
      self.__word_vocabulary, self.__char_vocabulary = self.__build_vocabs(path)

  def characters(self):
    return self.__char_vocabulary

  def words(self):
    return self.__word_vocabulary

  def save(self, path):
    with open(path, 'wb') as f:
      obj = {'vocab_size' : (self.word_vocab_size, self.char_vocab_size),
             'tag_to_index': self.tag_to_index,
             'unk_token' : self.unk_token,
             'words' : self.words(),
             'characters' : self.characters()}
      pickle.dump(obj, f)

  @staticmethod
  def load(path):
    with open(path, 'rb') as f:
      obj = pickle.load(f)
      return Vocab(obj['vocab_size'],
                   unk_token=obj['unk_token'], 
                   tag_to_index=obj['tag_to_index'],  
                   words=obj['words'], 
                   characters=obj['characters'])

  def __build_vocabs(self, path):
    char_counter = Counter()
    word_counter = Counter()
    # simplifying vocabulary with lemma (plurals and verbs)
    l = WordNetLemmatizer()
    with open(path, 'r') as file:
      for line in file:
        if line[0] == '#':
          line = line.split()[1:]
          
          #populating word vocabulary
          for word in line:
            word = l.lemmatize(l.lemmatize(word,'v'))
            word_counter[word.lower()] += 1
              
            #populating character vocabulary
            for char in word:
              char_counter[char] += 1
        
        else: continue

    word_vocabulary = {key:index for index, (key,_) in enumerate(word_counter.most_common(self.word_vocab_size-2))}
    word_vocabulary[self.unk_token] = self.word_vocab_size-2
    word_vocabulary['<pad>'] = self.word_vocab_size-1
    char_vocabulary = {key:index for index, (key,_) in enumerate(char_counter.most_common(self.char_vocab_size-1))}
    char_vocabulary[self.unk_token] = self.char_vocab_size-1

    return word_vocabulary, char_vocabulary

In [ ]:
folder = '/content/drive/My Drive/AIRO/NLP/Homework/homework1/'
data_path = 'nlp2020-hw1/data/'
vocab_size = (10000,50) #(word_vocab_size, char_vocab_size)

vocab = Vocab(vocab_size, path=folder+data_path+'train.tsv')

**Saving the Vocabulary**

In [ ]:
vocab.save(folder+'vocab-word+char.pickle')

### Pretrained Embedding
The choice of using pretrained embeddings is crucial. The choice was Global Vectors from Standford Wikipedia 2014 + Gigaword 5 with 6B tokens and 50 vector size representation.

In [ ]:
class GloVe():

  def __init__(self, path):

    self.vector_dim = 50
    self.path = path
    self.vectors = self.__load_vectors()

  def __contains__(self, item):
    return item in self.vectors

  def __getitem__(self, item):
    return self.vectors[item] if item in self.vectors else None
                   
  def __load_vectors(self):
    vectors_dict = dict()
    with open(self.path, 'r') as glove:
      for line in glove:
        values = line.split()
        word = values[0]
        vector = [float(i) for i in values[1:]]
        vectors_dict[word] = vector
    return vectors_dict

In [ ]:
glove = GloVe(folder+"glove.6B.50d.txt") 

### Utility
The Encoder class takes care of encoding samples in PyTorch Tensor for feed the neural networks. It also has methods for read text samples and decode output after prediction. For this, it makes use of a Vocab.

In [ ]:
class Encoder():

  device = 'cuda' if torch.cuda.is_available() else 'cpu'
  l = WordNetLemmatizer()
  # output of single layer of LSTM
  char_vec_dim = 50

  # sampling method for testing
  @staticmethod
  def read_test(sentence):
    sample = list()
    for w in sentence.split():      
      d = dict()
      d['word'] = w
      d['lemma'] = Encoder.l.lemmatize(Encoder.l.lemmatize(w,'v'))
      d['postag'] = '<test>'
      sample.append(d)
    return sample

  # return the index of the character in the vocabulary
  @staticmethod
  def __char_2_index(char, v:Vocab):
    if char in v.characters(): return v.characters()[char] 
    else: return v.characters()[v.unk_token]

  # returns the index of the word in the vocabulary
  @staticmethod
  def __word_2_index(word, v:Vocab):
    if word in v.words(): return v.words()[word]
    else: return v.words()[v.unk_token]

  @staticmethod
  def load_word_embeddings(v:Vocab, g:GloVe):
    embeddings = torch.randn(len(v.words()), g.vector_dim)
    for i, w in enumerate(v.words()):
      if w in g: embeddings[i] = torch.FloatTensor(g[w])
    return embeddings

  @staticmethod
  def load_char_embeddings(v:Vocab):
    embeddings = torch.randn(len(v.characters()), Encoder.char_vec_dim)
    # one hot encoding
    for i, w in enumerate(v.characters()):
      vector = [0]*Encoder.char_vec_dim
      vector[i] = 1
      embeddings[i] = torch.FloatTensor(vector)
    return embeddings

  """
    input:  list of dictionaries each per word in sentence
    output: vocabulary in which
      encoded_words  -> Tensor of [indexes of words in a sample] + <PAD>
      encoded_chars  -> lists of [Tensors of [indexes of chars of word] in a sample] 
      encoded_labels -> Tensor of [index of postag] + <PAD>
  """
  @staticmethod
  def encode(sample, v:Vocab):

    encoded_words = torch.LongTensor([Encoder.__word_2_index(w['lemma'].lower(), v) for w in sample]).to(Encoder.device)
  
    encoded_tags = torch.LongTensor([v.tag_to_index[w['postag']] for w in sample]).to(Encoder.device)
    
    encoded_chars = [torch.tensor([Encoder.__char_2_index(ch, v) for ch in w['word']]).to(Encoder.device) for w in sample]

    return {'words' : encoded_words, 'chars' : encoded_chars, 'tags' : encoded_tags}

  # Decoding the prediction
  @staticmethod
  def decode(indices, v:Vocab):
    # reverse tag_to_index mapping in order to check output indices
    index_to_tag = {v:k for k,v in v.tag_to_index.items()}
    indices = indices.squeeze().tolist()
    if type(indices) is list: return [index_to_tag[tag] for tag in indices]
    else: return [index_to_tag[indices]]

In [ ]:
sample = [{'word':'home','lemma':'home','postag':'O'},{'word':'bellissima','lemma':'bellissima','postag':'ORG'}, {'word':'Francesco','lemma':'Francesco','postag':'<test>'}]
print('[DEVICE]', Encoder.device)
encoded_sample = Encoder.encode(sample, vocab)
print(encoded_sample['words'].shape, encoded_sample['words'])
print(encoded_sample['chars'])
print(encoded_sample['tags'].shape, encoded_sample['tags'])

[DEVICE] cpu
torch.Size([3]) tensor([ 152, 9998, 8695])
[tensor([ 7,  3, 13,  0]), tensor([15,  0,  9,  9,  5,  8,  8,  5, 13,  2]), tensor([42,  6,  2,  4, 10,  0,  8, 10,  3])]
torch.Size([3]) tensor([0, 2, 8])


In [ ]:
# character representation of word with BiLSTM
def char_bilstm(word, v:Vocab):
  chars = torch.rand(len(word), 50)
  bilstm = nn.LSTM(50, 25, bidirectional=True, batch_first=True)
  _, (hidden, _) = bilstm(chars.unsqueeze(0))
  return torch.cat((hidden[0], hidden[1]),1)

char_bilstm('ciao', vocab)

tensor([[-0.0643, -0.1940, -0.1719, -0.1472, -0.0175, -0.1040,  0.1489, -0.1840,
          0.1471,  0.0731,  0.1022,  0.2541, -0.1660,  0.0427, -0.2378,  0.0442,
         -0.0412, -0.0569, -0.1550, -0.1837,  0.2991, -0.0095,  0.1728, -0.0198,
         -0.0535,  0.2710,  0.1536, -0.0456, -0.1888, -0.2756, -0.2404,  0.0736,
          0.2944, -0.1523,  0.3258, -0.0996,  0.2182, -0.0096, -0.0377,  0.0558,
         -0.2185,  0.1225,  0.0824, -0.0670,  0.0667, -0.2817,  0.0253,  0.1209,
          0.1276, -0.1604]], grad_fn=<CatBackward>)

### Dataset
The class organizes input text and samples it using a dictionary for each word 

```
{'word':'word', 'lemma':'lemma of word', 'postag':'tag of word'}
```
It is subclass of IterabaleDataset, where the __ iter __ method returns inputs and tags encoding the dictionary representation of each sentence.

In [ ]:
class MyDataset(IterableDataset):

  def __init__(self, path, vocab:Vocab):

    # max lengths of sample for padding
    self.max_s_len = 0
    self.vocab = vocab
    self.path = path
    self.__data = self.__build_dataset()

  def __len__(self):
    return len(self.__data)

  def __getitem__(self, index):
    return self.__data[index]

  """ 
    build list of lists of dictionaries
    sample --> [{word-1: _ , lemma-1: _ , postag-1: _ }...{word-n: _ , lemma-n: _ , postag-n: _ }]
  """
  def __build_dataset(self):
    data = list()
    with open(self.path, 'r') as file:

      for line in file:
        line = line.split()
        if len(line) == 0: continue
        elif line[0] == '#':
          count = len(line[1:])
          sample = list()

          for i in range(count):
            line = next(file).split()
            d = dict()
            d['word'] = line[1]
            d['lemma'] = Encoder.l.lemmatize(Encoder.l.lemmatize(line[1].lower(),'v'))
            d['postag'] = line[2]
            sample.append(d)

          if int(line[0]) > self.max_s_len: self.max_s_len = int(line[0])+1
          data.append(sample)

    return data

  # ecoded dataset 
  def __iter__(self):
    for i in range(len(self.__data)):
      yield Encoder.encode(self.__data[i], self.vocab)
      

In [ ]:
train_set = MyDataset(folder+data_path+'train.tsv', vocab)
dev_set = MyDataset(folder+data_path+'dev.tsv', vocab)

## Model

### Building
The Neural Network model is based on BiLSTM and make use of a dropout layer to prevent overfitting and finally returns a logits prediction where each one represent the probability of each class.


In [ ]:
class MyModel(nn.Module):

  def __init__(self, hparams):
    super(MyModel, self).__init__()

    # Word embedding
    self.word_embedding = nn.Embedding(hparams.word_vocab_size, hparams.word_embedding_dim)
    self.word_embedding.weight.data.copy_(hparams.word_embeddings)

    # Char embedding
    self.char_embedding = nn.Embedding(hparams.char_vocab_size, hparams.char_embedding_dim)
    self.char_embedding.weight.data.copy_(hparams.char_embeddings)

    # Bidirectional Char LSTM
    self.char_lstm = nn.LSTM(hparams.char_embedding_dim, hparams.char_hidden_size, 
                          batch_first=True,
                          bidirectional=True)

    # Bidirectional LSTM 
    self.lstm = nn.LSTM(hparams.embedding_dim, hparams.hidden_dim,
                        batch_first=True,
                        bidirectional=True)
    
    # dropout embedding
    self.dropout = nn.Dropout(hparams.dropout)
    
    # linear classifier
    self.classifier = nn.Linear(hparams.hidden_dim*2, hparams.num_tags)

  def forward(self, w, c):
    word_embedding = self.word_embedding(w)
    emb = []

    for i in range(len(w)):
      chars = torch.cat([self.char_embedding(ch).unsqueeze(0) for ch in c[i]])
      self.init_hidden(self.char_lstm)
      _, (hidden, _) = self.char_lstm(chars.unsqueeze(0))
      emb.append(torch.cat((hidden[0], hidden[1]),1).squeeze(0).tolist())
      
    char_embedding = torch.FloatTensor(emb).to(Encoder.device)
    embedding = torch.cat((word_embedding, char_embedding),1)
    x = self.dropout(embedding)
    o, (_, _) = self.lstm(x.unsqueeze(0))
    output = self.classifier(o)
    return output

  def init_hidden(self, lstm:nn.LSTM):
    for name, param in lstm.named_parameters():
      if 'bias' in name:
        nn.init.constant_(param, 0.0)
      elif 'weight' in name:
        nn.init.xavier_normal_(param)

### Hyperparameters

In [ ]:
class HParams():

  def __init__(self, v:Vocab, g:GloVe):

    # word
    self.word_vocab_size = v.word_vocab_size
    self.word_embedding_dim = g.vector_dim
    self.word_embeddings = Encoder.load_word_embeddings(v, g)

    # char
    self.char_vocab_size = v.char_vocab_size
    self.char_embedding_dim = Encoder.char_vec_dim
    self.char_hidden_size = int(self.char_embedding_dim/2)
    self.char_embeddings = Encoder.load_char_embeddings(v)

    # parameters
    self.embedding_dim = 100
    self.num_layers = 1
    self.hidden_dim = 128
    self.dropout = 0.2
    self.num_tags = len(v.tag_to_index)-2
    self.epochs = 5

params = HParams(vocab, glove)

## Training

In [ ]:
class Trainer():

  def __init__(self, model,loss_function, optimizer):
    self.model = model
    self.loss_function = loss_function
    self.optimizer = optimizer

  def train(self, train_dataset, valid_dataset, epochs=1):
    
    train_loss = 0.0
    
    for epoch in range(epochs):
      print(' Epoch {:03d}'.format(epoch + 1))

      # train mode
      self.model.train()

      epoch_loss = 0.0

      for step, sample in enumerate(train_dataset):
        words = sample['words']
        chars = sample['chars']
        tags = sample['tags']
        
        if step % 10 == 0: print('\r\t[training = {:0.2f} %]'.format(step*100/len(train_dataset)), end="")
        self.optimizer.zero_grad()
        predictions = self.model(words, chars)

        predictions = predictions.view(-1, predictions.shape[-1])
        tags = tags.view(-1)
  
        sample_loss = self.loss_function(predictions, tags)
        sample_loss.backward()

        #gradient clipping
        #clip_grad_value_(self.model.parameters(), 5)

        self.optimizer.step()

        epoch_loss += sample_loss.tolist()
      
      avg_epoch_loss = epoch_loss/len(train_dataset)
      train_loss += avg_epoch_loss
      print('\n\t[E: {:2d}] train loss = {:0.4f}'.format(epoch+1, avg_epoch_loss))

      valid_loss = self.evaluate(valid_dataset)
      print('\t[E: {:2d}] valid loss = {:0.4f}'.format(epoch+1, valid_loss))

    avg_epoch_loss = train_loss / epochs
    return avg_epoch_loss

  def evaluate(self, valid_dataset):
    valid_loss = 0.0
    
    # evaluation mode
    self.model.eval()

    with torch.no_grad():
      for sample in valid_dataset:

        words = sample['words']
        chars = sample['chars']
        tags = sample['tags']
        
        predictions = self.model(words, chars)
        predictions = predictions.view(-1, predictions.shape[-1])
        tags = tags.view(-1)
        sample_loss = self.loss_function(predictions, tags)

        valid_loss += sample_loss.tolist()

    return valid_loss / len(valid_dataset)

  def predict(self, x):
    # evaluation mode
    self.model.eval()
    with torch.no_grad():
      words = x['words']
      chars = x['chars']
      logits = self.model(words, chars)
      predictions = torch.argmax(logits, -1)
      return logits, predictions

In [ ]:
import time

start = time.time()
for step, sample in enumerate(train_set):
  if step == 10000: break
print(time.time()-start)

In [ ]:
postagger = MyModel(params).cuda() if Encoder.device == 'cuda' else MyModel(params)

trainer = Trainer(
    model = postagger,
    loss_function = nn.CrossEntropyLoss(),
    #optimizer = optim.SGD(postagger.parameters(), lr=0.01)
    optimizer = optim.Adam(postagger.parameters())
)

In [ ]:
trainer.train(train_set, dev_set, params.epochs)

## Evaluation
Accuracy it is not a good estimator. In fact, even with high accuray, we could have low precision (due to unbalanced sets).

In [ ]:
test_set = MyDataset(folder+data_path+'test.tsv', vocab)
test_set_loss = trainer.evaluate(test_set)
print("test set loss: {}".format(test_set_loss))

test set loss: 0.09673682880572643


In [ ]:
count, tot_samples = 0,0
tags = dict.fromkeys(vocab.tag_to_index, 0)
del tags['<test>']
del tags['<pad>']
matrix = dict()
for tag in tags:
  matrix[tag] = tags.copy()
precision = dict()
recall = dict()

for sample in test_set:

  test_y = sample['tags']
  
  _, predictions = trainer.predict(sample)
  
  x_tags = Encoder.decode(predictions,vocab)
  y_tags = Encoder.decode(test_y,vocab)

  for x, y in zip(x_tags, y_tags):
    tot_samples += 1
    if x == y: 
      count += 1
    matrix[x][y] +=1  

for key in matrix:
  precision[key] = matrix[key][key]/sum([matrix[key][k] for k in matrix])
  recall[key] = matrix[key][key]/sum([matrix[k][key] for k in matrix])

print('[ACCURACY] =',count/tot_samples)
print('[PRECISION]','[RECALL]','[F1 SCORE]')
for tag in precision:
  f1_score = 2*precision[tag]*recall[tag]/(precision[tag]+recall[tag])
  print('{}\t{:0.2f}\t{:0.2f}\t{:0.2f}'.format(tag, precision[tag], recall[tag], f1_score))


[ACCURACY] = 0.9706953341389599
[PRECISION] [RECALL] [F1 SCORE]
O	0.98	0.99	0.99
PER	0.91	0.88	0.90
ORG	0.81	0.64	0.72
LOC	0.84	0.82	0.83


Confusion Matrix

In [ ]:
import numpy as np
import matplotlib as plt
import seaborn as sns; sns.set()

confusion = dict()
for key in matrix:
  factor=1.0/sum(matrix[key].values())
  n = {k:v*factor for (k,v) in matrix[key].items()}
  confusion[key] = n

data = [np.array(list(d.values())) for d in confusion.values()]
ax = sns.heatmap(data, vmin=0, vmax=0.2, square=True, cmap='coolwarm', annot=True)

## Prediction
A prediction method for testing every single sentence

In [ ]:
sentence = 'Francesco went with Lorenzo to the airport in Rome'
sample = Encoder.read_test(sentence)
encoded_sample = Encoder.encode(sample, vocab)
_, predictions = trainer.predict(encoded_sample)
Encoder.decode(predictions, vocab)

**Saving the Model**

In [ ]:
torch.save(postagger.state_dict(), folder+'postagger-word+char.txt')